In [1]:
%reload_ext autoreload
%autoreload 2

In [108]:
url

'http://127.0.0.1:5543/generate_stream'

In [110]:
import sseclient

obj = {
        "inputs": sequence,
        "generation_parameters": {
            "max_new_tokens":100,
            # "repetion_penalty": 2.0,
            # "do_sample": False,
            # "temperature": 1.0,
            # "top_k": 1,
            # "top_p": 0.9,
            # "seed": 43,
        }
    }

stream_response = requests.post(url, json=obj, stream=True)

client = sseclient.SSEClient(stream_response)

# Loop forever (while connection "open")
for event in client.events():
    print ("got a new event from server")
    pprint.pprint(event.data)

KeyboardInterrupt: 

In [95]:
!pip install sseclient

  Preparing metadata (setup.py) ... done
  Created wheel for sseclient: filename=sseclient-0.0.27-py3-none-any.whl size=5565 sha256=56a8fb4039994d2facce58dba0bad7a3af7f5a9569e80b70ec2adab94dc46a35
  Stored in directory: /home/robertgshaw/.cache/pip/wheels/73/21/3f/9d20c05c1c9312ab61bc7bf63bb2c189a71e800cb028fa8d2a
Successfully built sseclient


In [96]:
import sseclient, pprint

stream_response = requests.get(url, json=obj, stream=True)
client = sseclient.SSEClient(stream_response)

# Loop forever (while connection "open")
for event in client.events():
    print ("got a new event from server")
    pprint.pprint(event.data)

InvalidURL: Failed to parse: <Response [405]>

In [93]:
import requests
from threading import Thread
import json

url = "http://127.0.0.1:5543/generate_stream"
sequence = "Finish the following function for computing a fibonacci sequence: \n\ndef fib(n):"
# sequence = "def fib(n):"

def request_task(seed, max_new_tokens):
    obj = {
        "inputs": sequence,
        "generation_parameters": {
            "max_new_tokens":max_new_tokens,
            # "repetition_penalty": 1.1,
            # "do_sample": True,
            # "temperature": 1.1,
            # "top_k": 3,
            # "top_p": 0.9,
            "seed": seed,
        }
    }
    with requests.post(url, json=obj) as r:
        print(max_new_tokens)
        dct = json.loads(r.text)
        print(dct)
        print(f'{sequence}{dct["response_text"]}')

max_new_tokens_lst = [55, 50, 45]
seeds = [1,2,3]
# max_new_tokens_lst = [100, 200, 300]

cnt = 1
request_ts = [
    Thread(target=request_task, args=[seed, max_new_tokens]) 
    for seed, max_new_tokens in zip(seeds[:cnt], max_new_tokens_lst[:cnt])
]

import time
for request_t in request_ts:
    request_t.start()
    time.sleep(0.1)

for request_t in request_ts:
    request_t.join()

Exception in thread Thread-179:
Traceback (most recent call last):
  File "/home/robertgshaw/.conda/envs/dscb/lib/python3.9/threading.py", line 980, in _bootstrap_inner
    self.run()
  File "/home/robertgshaw/.conda/envs/dscb/lib/python3.9/threading.py", line 917, in run
    self._target(*self._args, **self._kwargs)
  File "/tmp/ipykernel_9240/1133841396.py", line 26, in request_task
TypeError: 'NoneType' object is not subscriptable


55
None


In [36]:
obj = {
        "inputs": sequence,
        "generation_parameters": {
            "max_new_tokens":100,
            "repetion_penalty": 2.0,
            # "do_sample": False,
            # "temperature": 1.0,
            # "top_k": 1,
            # "top_p": 0.9,
            # "seed": 43,
        }
    }

resp = requests.post(url, json=obj)

In [47]:
a = ""
token = '\n'
for i in range(5):
    a += token

In [48]:
print(a)

In [44]:
print(resp.text)

"\n    if n == 0:\n        return 0\n    elif n == 1:\n        return 1\n    else:\n        return fib(n-1) + fib(n-2)\n\ndef fib2(n):\n    if n == 0:\n        return 0\n    elif n == 1:\n        return 1\n    else:\n        return fib2(n-1) + fib2(n-2)\n\ndef"


In [1]:
%cd deepsparse

/home/robertgshaw/deepsparse-continuous-batching/deepsparse


In [5]:
import numpy as np
from logits_process import TopKLogitsWarper, TopPLogitsWarper, softmax

scores = np.array([[-1.,1.,4.,3.,2.,1.]])

sorted_indices = np.argsort(scores, axis=-1)
sorted_scores = np.take_along_axis(scores, sorted_indices, axis=-1)

# sort, grabbing all those outside top_p
cumulative_probs = softmax(sorted_scores, axis=-1).cumsum(axis=-1)
sorted_indices_to_remove = cumulative_probs <= (1 - 0.9)


# note: this relies on b=1
indices_to_remove = sorted_indices[sorted_indices_to_remove]

# set removed indices logits to -Inf (never selected)
scores[:, indices_to_remove] = -float("Inf")

[[-1.  1.  4.  3.  2.  1.]]


In [93]:
from logits_process import TopKLogitsWarper, TopPLogitsWarper, softmax

logits = np.array([[-2.,1.,5.,3.,2.,1.], [5.,-2.,3.,-1.,2.,1.]])
print(logits)

top_k = TopKLogitsWarper(top_k=3)
new_logits = top_k(logits)
print(new_logits)

print(softmax(new_logits, axis=1))

[[-2.  1.  5.  3.  2.  1.]
 [ 5. -2.  3. -1.  2.  1.]]
[[-inf -inf   5.   3.   2. -inf]
 [  5. -inf   3. -inf   2. -inf]]
[[0.         0.         0.84379473 0.1141952  0.04201007 0.        ]
 [0.84379473 0.         0.1141952  0.         0.04201007 0.        ]]


In [97]:
logits = np.array([[-1.,1.,4.,3.,2.,1.]])
print(logits)

print(softmax(logits, axis=-1))

top_p = TopPLogitsWarper(top_p=0.9)
new_logits = top_p(logits)
print(new_logits)

print(softmax(new_logits, axis=-1))

[[-1.  1.  4.  3.  2.  1.]]
[[0.00418629 0.03093274 0.62130062 0.22856372 0.0840839  0.03093274]]
[[-inf -inf   4.   3.   2. -inf]]
[[0.         0.         0.66524096 0.24472847 0.09003057 0.        ]]


In [102]:
logits = np.array([[-1.,1.,3.,3.5,2.5,2.,1.]])
print(logits)

print(softmax(logits, axis=-1))

top_p = TopPLogitsWarper(top_p=0.9)
top_k = TopKLogitsWarper(top_k=3)

new_logits = top_p(logits)
print(new_logits)

new_new_logits = top_k(new_logits)
print(new_new_logits)
print(softmax(logits, axis=-1))

[[-1.   1.   3.   3.5  2.5  2.   1. ]]
[[0.00468177 0.03459387 0.25561604 0.4214396  0.15503896 0.09403589
  0.03459387]]
[[-inf -inf  3.   3.5  2.5  2.  -inf]]
[[-inf -inf  3.   3.5  2.5 -inf -inf]]
[[0.         0.         0.30719589 0.50648039 0.18632372 0.
  0.        ]]


In [113]:
logits = np.array([[-1.,1.,3.,5,2.5,2.,1.]])
print(logits)
print(softmax(logits, axis=-1))

top_p = TopPLogitsWarper(top_p=0.99)
top_k = TopKLogitsWarper(top_k=3)

new_logits = top_p(logits)
print(new_logits)

new_new_logits = top_k(new_logits)
print(new_new_logits)
# print(softmax(new_new_logits, axis=-1))

[[-1.   1.   3.   5.   2.5  2.   1. ]]
[[0.00189751 0.01402082 0.10360061 0.76551075 0.06283695 0.03811254
  0.01402082]]
[[-inf  1.   3.   5.   2.5  2.   1. ]]
[[-inf -inf  3.   5.   2.5 -inf -inf]]


In [12]:
cumulative_probs <= (1 - top_p)

array([[False, False, False, False, False]])

In [6]:
sorted_indices

array([[0, 4, 3, 1, 2]])

In [65]:
from utils import NextTokenChooser

# picks largest logits
ntc = NextTokenChooser(do_sample=False)
logits = np.array([[-2,1,3,2]])
input_ids = np.array([[1,2,3,4]])
ntc(input_ids, logits)

ModuleNotFoundError: No module named 'utils'

In [53]:
# samples
ntc = NextTokenChooser(do_sample=True)
logits = np.array([[-2,1,3,2,1]])
input_ids = np.array([[1,2,3,4]])

probs = ntc.choice.softmax(logits)
print(probs)

iters = 10000
counts = {a: 0 for a in range(logits.shape[1])}

for _ in range(iters):
    pred = ntc(input_ids, logits)
    counts[pred] += 1

for i in range(logits.shape[1]):
    print(f"sample / actual {i}: {counts[i] / iters: 0.3f} / {probs[0,i]: 0.3f}")

[[0.0040953  0.08225629 0.60779634 0.22359578 0.08225629]]
sample / actual 0:  0.004 /  0.004
sample / actual 1:  0.081 /  0.082
sample / actual 2:  0.615 /  0.608
sample / actual 3:  0.219 /  0.224
sample / actual 4:  0.081 /  0.082


In [57]:
# should pick logits that are less likely
ntc = NextTokenChooser(do_sample=True, temperature=3.0)
logits = np.array([[-2,1,3,2,1]])
input_ids = np.array([[1,2,3,4]])

probs = ntc.choice.softmax(logits)
print(probs)

iters = 10000
counts = {a: 0 for a in range(logits.shape[1])}

for _ in range(iters):
    pred = ntc(input_ids, logits)
    counts[pred] += 1

for i in range(logits.shape[1]):
    print(f"sample / actual {i}: {counts[i] / iters: 0.3f} / {probs[0,i]: 0.3f}")

[[0.0040953  0.08225629 0.60779634 0.22359578 0.08225629]]
sample / actual 0:  0.066 /  0.004
sample / actual 1:  0.176 /  0.082
sample / actual 2:  0.342 /  0.608
sample / actual 3:  0.244 /  0.224
sample / actual 4:  0.172 /  0.082


In [61]:
# should approach uniform
ntc = NextTokenChooser(do_sample=True, temperature=100.0)
logits = np.array([[-2,1,3,2,1]])
input_ids = np.array([[1,2,3,4]])

probs = ntc.choice.softmax(logits)
print(probs)

iters = 10000
counts = {a: 0 for a in range(logits.shape[1])}

for _ in range(iters):
    pred = ntc(input_ids, logits)
    counts[pred] += 1

for i in range(logits.shape[1]):
    print(f"sample / actual {i}: {counts[i] / iters: 0.3f} / {probs[0,i]: 0.3f}")

[[0.0040953  0.08225629 0.60779634 0.22359578 0.08225629]]
sample / actual 0:  0.197 /  0.004
sample / actual 1:  0.201 /  0.082
sample / actual 2:  0.205 /  0.608
sample / actual 3:  0.203 /  0.224
sample / actual 4:  0.195 /  0.082


In [64]:
# should pick logits that are more likely
ntc = NextTokenChooser(do_sample=True, temperature=0.5)
logits = np.array([[-2,1,3,2,1]])
input_ids = np.array([[1,2,3,4]])

probs = ntc.choice.softmax(logits)
print(probs)

iters = 10000
counts = {a: 0 for a in range(logits.shape[1])}

for _ in range(iters):
    pred = ntc(input_ids, logits)
    counts[pred] += 1

for i in range(logits.shape[1]):
    print(f"sample / actual {i}: {counts[i] / iters: 0.3f} / {probs[0,i]: 0.3f}")

[[0.0040953  0.08225629 0.60779634 0.22359578 0.08225629]]
sample / actual 0:  0.000 /  0.004
sample / actual 1:  0.015 /  0.082
sample / actual 2:  0.857 /  0.608
sample / actual 3:  0.111 /  0.224
sample / actual 4:  0.017 /  0.082


In [65]:
# should approximate greedy sampling
ntc = NextTokenChooser(do_sample=True, temperature=0.001)
logits = np.array([[-2,1,3,2,1]])
input_ids = np.array([[1,2,3,4]])

probs = ntc.choice.softmax(logits)
print(probs)

iters = 10000
counts = {a: 0 for a in range(logits.shape[1])}

for _ in range(iters):
    pred = ntc(input_ids, logits)
    counts[pred] += 1

for i in range(logits.shape[1]):
    print(f"sample / actual {i}: {counts[i] / iters: 0.3f} / {probs[0,i]: 0.3f}")

[[0.0040953  0.08225629 0.60779634 0.22359578 0.08225629]]
sample / actual 0:  0.000 /  0.004
sample / actual 1:  0.000 /  0.082
sample / actual 2:  1.000 /  0.608
sample / actual 3:  0.000 /  0.224
sample / actual 4:  0.000 /  0.082


In [134]:
# should approximate greedy sampling
ntc = NextTokenChooser(do_sample=False)

# should select index 4
logits = np.array([[-2.,1.,3.,2.,5.]])
input_ids = np.array([[3,4]])

print(ntc(input_ids,logits))

# should select index 4
ntc = NextTokenChooser(do_sample=False, repetition_penalty=1.0)
logits = np.array([[-2,1,3,2,5]])
input_ids = np.array([[3,4]])
print(ntc(input_ids,logits))

# should select index 2
ntc = NextTokenChooser(do_sample=False, repetition_penalty=2.0)
logits = np.array([[-2.,1.,3.,2.,5.]])
input_ids = np.array([[3,4]])
print(ntc(input_ids,logits))

# should select index 4
logits = np.array([[-2.,1.,3.,2.,5.]])
input_ids = np.array([[2,4]])
ntc = NextTokenChooser(do_sample=False, repetition_penalty=2.)
print(ntc(input_ids,logits))

# should select index 3
logits = np.array([[-2.,1.,3.,2.,5.]])
input_ids = np.array([[2,4]])
ntc = NextTokenChooser(do_sample=False, repetition_penalty=3.)
print(ntc(input_ids,logits))

# should select index 1
logits = np.array([[-2.,1.,3.,2.,5.]])
input_ids = np.array([[2,3,4]])
ntc = NextTokenChooser(do_sample=False, repetition_penalty=5.)
print(ntc(input_ids,logits))


# should make 2,4 less liekly
logits_og = np.array([[-1.,1.,3.,2.,4.]])
input_ids = np.array([[2,4]])
ntc = NextTokenChooser(do_sample=True, repetition_penalty=2.)

original_ps = ntc.choice.softmax(logits_og,axis=1)
penalty = np.array([[1.,1.,2.,1.,2.]])
penalty_ps = ntc.choice.softmax(logits_og/penalty,axis=1)

iters = 20000
counts = {a: 0 for a in range(logits.shape[1])}

for _ in range(iters):
    logits = logits_og.copy()
    # print(logits)
    pred = ntc(input_ids, logits)
    counts[pred] += 1

for i in range(logits.shape[1]):
    print(f"sample / actual / original {i}: {counts[i] / iters: 0.3f} / {penalty_ps[0,i]: 0.3f} / {original_ps[0,i]: 0.3f}")

4
4
2
4
3
1
sample / actual / original 0:  0.017 /  0.016 /  0.004
sample / actual / original 1:  0.119 /  0.122 /  0.032
sample / actual / original 2:  0.204 /  0.201 /  0.236
sample / actual / original 3:  0.329 /  0.331 /  0.087
sample / actual / original 4:  0.330 /  0.331 /  0.641


In [111]:
logits / penalty

array([[-2. ,  1. ,  1.5,  2. ,  2.5]])

In [24]:
from utils import Sampling
import numpy as np

sampling = Sampling(seed=1)

logits = np.array([[-2,1,2,3]])
probs = sampling.softmax(logits, axis=1)

print(probs)
counts = {a: 0 for a in range(logits.shape[1])}
iters = 10000
for i in range(iters):
    counts[sampling(logits=logits)] +=1

for key in counts:
    counts[key] /= iters

print(counts)

[[0.00446236 0.08962882 0.24363641 0.66227241]]
{0: 0.0041, 1: 0.0909, 2: 0.2418, 3: 0.6632}


In [42]:
import numpy as np

class RepetitionPenaltyLogitsProcessor:
    def __init__(self, penalty: float):
        if not isinstance(penalty, float) or not (penalty > 0):
            raise ValueError(f"`penalty` has to be a strictly positive float, but is {penalty}")

        self.penalty = penalty

    def __call__(self, scores: np.ndarray, input_ids: np.ndarray) -> np.ndarray:
        # assert shape is [1, vocab_size]
        assert len(scores.shape) == 2
        assert scores.shape[0] == 1

        # assert shape is [1, seq_len]
        assert len(input_ids.shape) == 2
        assert input_ids.shape[0] == 1
        
        # TODO: update logic to handle b > 1
        score = scores[:, input_ids[0]]
        score = np.where(score < 0, score * self.penalty, score / self.penalty)
        scores[:, input_ids[0]] = score

        return scores

class TemperatureLogitsWarper:
    def __init__(self, temperature: float):
        if not isinstance(temperature, float) or not (temperature > 0):
            except_msg = (
                f"`temperature` (={temperature}) has to be a strictly positive float, otherwise your next token "
                "scores will be invalid."
            )
            if isinstance(temperature, float) and temperature == 0.0:
                except_msg += " If you're looking for greedy decoding strategies, set `do_sample=False`."
            raise ValueError(except_msg)
        
        self.temperature = temperature

    def __call__(self, scores: np.ndarray) -> np.ndarray:
        # assert shape is [1, vocab_size]
        assert len(scores.shape) == 2
        assert scores.shape[0] == 1

        return scores / self.temperature

input_ids = np.array([[0,2,5,7]])
logits = np.array([[0.01, -0.01]*4])

print(logits)
print(input_ids)

processor = RepetitionPenaltyLogitsProcessor(penalty=2.0)
logits = processor(scores=logits, input_ids=input_ids)
print(logits)

warper = TemperatureLogitsWarper(temperature=2.0)
logits = warper(scores=logits)
print(logits)

[[ 0.01 -0.01  0.01 -0.01  0.01 -0.01  0.01 -0.01]]
[[0 2 5 7]]
[[ 0.005 -0.01   0.005 -0.01   0.01  -0.02   0.01  -0.02 ]]
[[ 0.0025 -0.005   0.0025 -0.005   0.005  -0.01    0.005  -0.01  ]]


p# **Interacting with FastAPI Server**

In [17]:
!curl 127.0.0.1:5543/generate \
    -X POST \
    -d '{"inputs":"Finish the following function for computing a fibonacci sequence: \n\ndef fib(n):","generation_parameters":{"max_new_tokens":10}}' \
    -H 'Content-Type: application/json'

"Finish the following function for computing a fibonacci sequence: \n\ndef fib(n):\n    if n == 0:\n        "

In [20]:
import requests
from threading import Thread

url = "http://127.0.0.1:5543/generate"
# sequence = "Finish the following function for computing a fibonacci sequence: \n\ndef fib(n):"
sequence = "def fib(n):"

def request_task(max_new_tokens):
    obj = {
        "inputs":sequence,
        "generation_parameters": {
            "max_new_tokens":max_new_tokens
        }
    }
    with requests.post(url, json=obj) as r:
        print(max_new_tokens)
        print(r.text)


In [21]:
# max_new_tokens_lst = [50, 10, 100, 25, 15]
max_new_tokens_lst = [100, 200, 300]

request_ts = [
    Thread(target=request_task, args=[max_new_tokens]) for max_new_tokens in max_new_tokens_lst
]

import time
for request_t in request_ts:
    request_t.start()
    time.sleep(0.1)

for request_t in request_ts:
    request_t.join()

100
"def fib(n):\n    if n == 0:\n        return 0\n    elif n == 1:\n        return 1\n    else:\n        return fib(n-1) + fib(n-2)\n\ndef fib2(n):\n    if n == 0:\n        return 0\n    elif n == 1:\n        return 1\n    else:\n        return fib2(n-1) + fib2(n-2)\n\ndef"
200
"def fib(n):\n    if n == 0:\n        return 0\n    elif n == 1:\n        return 1\n    else:\n        return fib(n-1) + fib(n-2)\n\ndef fib2(n):\n    if n == 0:\n        return 0\n    elif n == 1:\n        return 1\n    else:\n        return fib2(n-1) + fib2(n-2)\n\ndef fib3(n):\n    if n == 0:\n        return 0\n    elif n == 1"
300
"def fib(n):\n    if n == 0:\n        return 0\n    elif n == 1:\n        return 1\n    else:\n        return fib(n-1) + fib(n-2)\n\ndef fib2(n):\n    if n == 0:\n        return 0\n    elif n == 1:\n        return 1\n    else:\n        return fib2(n-1) + fib2(n-2)\n\ndef fib3(n):\n    if n == 0:\n        return 0\n    elif n == 1"


In [4]:
!curl 127.0.0.1:5543/generate_stream \
    -X POST \
    -d '{"prompt":"Finish the following function for computing a fibonacci sequence: \n\ndef fib(n):","max_generated_tokens":100}' \
    -H 'Content-Type: application/json'


    if n == 0:
        return 0
    elif n == 1:
        return 1
    else:
        return fib(n-1) + fib(n-2)

# Driver function to test above function
n = int(input("Enter the number: "))
print(fib(n))

# This code is contributed by Nikhil Kumar Singh(nickzuck_007)


In [13]:
!curl http://127.0.0.1:5543/generate/def%20fib%28n%29%3A

"def fib(n):\n    if n == 0:\n        return 0\n    elif n == 1:\n        return 1\n    else:\n        return fib(n-1) + fib(n-2)\n\ndef fib2(n):\n    if n == 0:\n        return 0\n    elif n == 1:\n        return 1\n    else:\n        return fib2(n-1) + fib2(n-2)\n\ndef fib3(n):\n    if n == 0:\n        return 0\n    elif n == 1"

## Example Interacting With The Router

In [3]:
from server.deepsparse.router import DeepSparseRouter, batching_task
from server.deepsparse.service.service import DeepSparseService
from server.deepsparse.service.causal_lm import DeepSparseCausalLM

2023-08-24 17:46:45 deepsparse.transformers WARNING  The neuralmagic fork of transformers may not be installed. It can be installed via `pip install nm_transformers`


In [4]:
tokenizer_path = "/home/robertgshaw/.cache/sparsezoo/neuralmagic/codegen_mono-350m-bigpython_bigquery_thepile-base/deployment"
onnx_path = "/home/robertgshaw/.cache/sparsezoo/neuralmagic/codegen_mono-350m-bigpython_bigquery_thepile-base/model.onnx/model.onnx"

model = DeepSparseCausalLM(
    tokenizer_path=tokenizer_path,
    model_path=onnx_path
)

Using pad_token, but it is not set yet.
2023-08-24 17:47:02 deepsparse.transformers.utils.helpers INFO     Overwriting in-place the input shapes of the transformer model at /home/robertgshaw/.cache/sparsezoo/neuralmagic/codegen_mono-350m-bigpython_bigquery_thepile-base/model.onnx/model.onnx
DeepSparse, Copyright 2021-present / Neuralmagic, Inc. version: 1.6.0.20230815 COMMUNITY | (134dba40) (release) (optimized) (system=avx2, binary=avx2)


deepsparse.engine.Engine:
	onnx_file_path: /home/robertgshaw/.cache/sparsezoo/neuralmagic/codegen_mono-350m-bigpython_bigquery_thepile-base/model.onnx/model.onnx
	batch_size: 1
	num_cores: 8
	num_streams: 1
	scheduler: Scheduler.default
	fraction_of_supported_ops: 1.0
	cpu_avx_type: avx2
	cpu_vnni: False


2023-08-24 17:47:27 deepsparse.transformers.utils.helpers INFO     Overwriting in-place the input shapes of the transformer model at /home/robertgshaw/.cache/sparsezoo/neuralmagic/codegen_mono-350m-bigpython_bigquery_thepile-base/model.onnx/model.onnx


deepsparse.engine.Engine:
	onnx_file_path: /home/robertgshaw/.cache/sparsezoo/neuralmagic/codegen_mono-350m-bigpython_bigquery_thepile-base/model.onnx/model.onnx
	batch_size: 1
	num_cores: 8
	num_streams: 1
	scheduler: Scheduler.default
	fraction_of_supported_ops: 1.0
	cpu_avx_type: avx2
	cpu_vnni: False


In [6]:
from server.deepsparse.utils import GenerateRequest
from queue import Queue 
from threading import Thread
import time

service = DeepSparseService(model=model)
router = DeepSparseRouter(service=service)
batching_thread = Thread(target=batching_task, args=[router])
batching_thread.start()

In [8]:
prompts = [
    "Finish the following function for computing a fibonacci sequence: \n\ndef fib(n):",
    "Write a function for filtering a list of integers to include only positive numbers:\n\ndef filter(lst):",
    "Write a function for reversing a string:\n\ndef reverse_string(s):",
    "Write a function for checking if a word if a palindrome:\n\ndef is_palindrome(word):",
    "Write a function for sorting an array of integers:\n\ndef merge_sort(arr):",
]

def generate_request(prompt):
    print("starting request")
    response_stream = Queue()
    
    g_request = GenerateRequest(
        prompt=prompt,
        max_generated_tokens=100,
        response_stream=response_stream
    )

    router.generate(g_request)

    str = prompt
    generation = response_stream.get()
    while not generation.stopped:
        str += generation.token
        generation = response_stream.get()

    print(str)

generate_threads = [
    Thread(target=generate_request, args=[prompt]) for prompt in prompts[:2]
]

# print(len(generate_threads))

for gt in generate_threads:
    gt.start()
    time.sleep(1)

for gt in generate_threads:
    gt.join()

starting request
starting request
Finish the following function for computing a fibonacci sequence: 

def fib(n):
    if n == 0:
        return 0
    elif n == 1:
        return 1
    else:
        return fib(n-1) + fib(n-2)

# Driver function to test above function
n = int(input("Enter the number: "))
print(fib(n))

# This code is contributed by Nikhil Kumar Singh(nickzuck_007)

Write a function for filtering a list of integers to include only positive numbers:

def filter(lst):
    return [x for x in lst if x > 0]

# Test
print(filter([1, 2, 3, 4, 5, 6, 7, 8, 9, 10]))
print(filter([1, 2, 3, 4, 5, 6, 7, 8, 9, 10]))
print(filter([1, 2, 3, 4, 5, 6, 7, 8, 9, 10]))
print(filter([1,


In [9]:
service = DeepSparseService(model=model)
router = DeepSparseRouter(service=service)
batching_thread.join()

KeyboardInterrupt: 

In [54]:
batch, generate_requests = router.queue.next_batch(block=True)

In [55]:
cached_batch = router.prefill(batch, generate_requests)
print(cached_batch)

CachedBatch(batch_id=0, request_ids=[0])


In [56]:
next_batch = router.queue.next_batch(block=False)
print(next_batch)

None


In [57]:
for i in range(20):
    cached_batch = router.decode(batches=[cached_batch], generate_requests=generate_requests)
    print(cached_batch)

CachedBatch(batch_id=0, request_ids=[0])
CachedBatch(batch_id=0, request_ids=[0])
CachedBatch(batch_id=0, request_ids=[0])
CachedBatch(batch_id=0, request_ids=[0])
CachedBatch(batch_id=0, request_ids=[0])
CachedBatch(batch_id=0, request_ids=[0])
CachedBatch(batch_id=0, request_ids=[0])
CachedBatch(batch_id=0, request_ids=[0])
CachedBatch(batch_id=0, request_ids=[0])
CachedBatch(batch_id=0, request_ids=[0])
CachedBatch(batch_id=0, request_ids=[0])
CachedBatch(batch_id=0, request_ids=[0])
CachedBatch(batch_id=0, request_ids=[0])
CachedBatch(batch_id=0, request_ids=[0])
CachedBatch(batch_id=0, request_ids=[0])
CachedBatch(batch_id=0, request_ids=[0])
CachedBatch(batch_id=0, request_ids=[0])
CachedBatch(batch_id=0, request_ids=[0])
CachedBatch(batch_id=0, request_ids=[0])
CachedBatch(batch_id=0, request_ids=[0])


In [58]:
router.generate(g_request1)

In [59]:
next_batch = router.queue.next_batch(block=False)
if next_batch is not None:
    new_batch, new_generate_requests = next_batch

new_cached_batch = router.prefill(
    batch=new_batch,
    generate_requests=new_generate_requests
)

In [60]:
batches = [cached_batch]
batches.append(new_cached_batch)
generate_requests.update(new_generate_requests)

In [63]:
for i in range(20):
    cached_batch = router.decode(batches=batches, generate_requests=generate_requests)
    batches = [cached_batch]

In [62]:
generate_requests

{0: GenerateRequest(prompt='Finish the following function for computing a fibonacci sequence: \n\ndef fib(n):', max_generated_tokens=100, response_stream=<queue.Queue object at 0x7f552c1b3910>),
 1: GenerateRequest(prompt='Write a function for reversing a string:\n\ndef reverse_string(s):', max_generated_tokens=100, response_stream=<queue.Queue object at 0x7f5557c30ac0>)}

In [66]:
for i in range(20):
    generation = response_stream1.get(block=False)
    print(generation)

Generation(request_id=1, token='\n', token_id=198, stopped=False)
Generation(request_id=1, token='    ', token_id=50284, stopped=False)
Generation(request_id=1, token='return', token_id=7783, stopped=False)
Generation(request_id=1, token=' s', token_id=264, stopped=False)
Generation(request_id=1, token='[', token_id=58, stopped=False)
Generation(request_id=1, token='::', token_id=3712, stopped=False)
Generation(request_id=1, token='-', token_id=12, stopped=False)
Generation(request_id=1, token='1', token_id=16, stopped=False)
Generation(request_id=1, token=']', token_id=60, stopped=False)
Generation(request_id=1, token='\n', token_id=198, stopped=False)
Generation(request_id=1, token='\n', token_id=198, stopped=False)
Generation(request_id=1, token='#', token_id=2, stopped=False)
Generation(request_id=1, token=' Test', token_id=6208, stopped=False)
Generation(request_id=1, token='\n', token_id=198, stopped=False)
Generation(request_id=1, token='print', token_id=4798, stopped=False)
Gen

In [5]:
from threading import Thread
import time

batching_thread = Thread(target=batching_task, args=[router])
batching_thread.start()

prompts = [
    "Finish the following function for computing a fibonacci sequence: \n\ndef fib(n):",
    "Write a function for filtering a list of integers to include only positive numbers:\n\ndef filter(lst):",
    "Write a function for reversing a string:\n\ndef reverse_string(s):",
    "Write a function for checking if a word if a palindrome:\n\ndef is_palindrome(word):",
    "Write a function for sorting an array of integers:\n\ndef merge_sort(arr):",
]

def generate_task(prompt):
    result = router.generate(prompt=prompt)
    print(result)
    print("\n")

generate_threads = [
    Thread(target=generate_task, args=[prompt]) for prompt in prompts
]

# print(len(generate_threads))

for gt in generate_threads:
    gt.start()
    time.sleep(0.5)

for gt in generate_threads:
    gt.join()


generate("stop")
batching_thread.join()

Finish the following function for computing a fibonacci sequence: 

def fib(n):
    if n == 0:
        return 0
    elif n == 1:
        return 1
    else:
        return fib(n-1) + fib(n-2)

# Driver function to test above function
n = int(input("Enter the number: "))
print(fib(n))

# This code is contributed by Nikhil Kumar Singh(nickzuck_007)



Write a function for filtering a list of integers to include only positive numbers:

def filter(lst):
    return [x for x in lst if x > 0]

# Test
print(filter([1, 2, 3, 4, 5, 6, 7, 8, 9, 10]))
print(filter([1, 2, 3, 4, 5, 6, 7, 8, 9, 10]))
print(filter([1, 2, 3, 4, 5, 6, 7, 8, 9, 10]))
print(filter([1,


Write a function for checking if a word if a palindrome:

def is_palindrome(word):
    return word == word[::-1]

# Test
print(is_palindrome("racecar"))
print(is_palindrome("racecar"))
print(is_palindrome("racecar"))
print(is_palindrome("racecar"))
print(is_palindrome("racecar"))
print(is_palindrome("racecar"))
print(is_palindrome("racecar"

## Example Interacting With The Service

In [2]:
from server.deepsparse.deepsparse_causal_lm import DeepSparseCausalLMBatch, DeepSparseCausalLM
from server.deepsparse.deepsparse_service import DeepSparseService
from server.deepsparse.deepsparse_requests import (
    PrefillRequest, DecodeRequest, FilterBatchRequest, Request
)

2023-08-22 14:26:39 deepsparse.transformers WARNING  The neuralmagic fork of transformers may not be installed. It can be installed via `pip install nm_transformers`


In [3]:
tokenizer_path = "/home/robertgshaw/.cache/sparsezoo/neuralmagic/codegen_mono-350m-bigpython_bigquery_thepile-base/deployment"
onnx_path = "/home/robertgshaw/.cache/sparsezoo/neuralmagic/codegen_mono-350m-bigpython_bigquery_thepile-base/model.onnx/model.onnx"

model = DeepSparseCausalLM(
    tokenizer_path=tokenizer_path,
    model_path=onnx_path
)

service = DeepSparseService(model=model)

Using pad_token, but it is not set yet.
2023-08-22 14:26:56 deepsparse.transformers.utils.helpers INFO     Overwriting in-place the input shapes of the transformer model at /home/robertgshaw/.cache/sparsezoo/neuralmagic/codegen_mono-350m-bigpython_bigquery_thepile-base/model.onnx/model.onnx
DeepSparse, Copyright 2021-present / Neuralmagic, Inc. version: 1.6.0.20230815 COMMUNITY | (134dba40) (release) (optimized) (system=avx2, binary=avx2)


deepsparse.engine.Engine:
	onnx_file_path: /home/robertgshaw/.cache/sparsezoo/neuralmagic/codegen_mono-350m-bigpython_bigquery_thepile-base/model.onnx/model.onnx
	batch_size: 1
	num_cores: 8
	num_streams: 1
	scheduler: Scheduler.default
	fraction_of_supported_ops: 1.0
	cpu_avx_type: avx2
	cpu_vnni: False


2023-08-22 14:27:21 deepsparse.transformers.utils.helpers INFO     Overwriting in-place the input shapes of the transformer model at /home/robertgshaw/.cache/sparsezoo/neuralmagic/codegen_mono-350m-bigpython_bigquery_thepile-base/model.onnx/model.onnx


deepsparse.engine.Engine:
	onnx_file_path: /home/robertgshaw/.cache/sparsezoo/neuralmagic/codegen_mono-350m-bigpython_bigquery_thepile-base/model.onnx/model.onnx
	batch_size: 1
	num_cores: 8
	num_streams: 1
	scheduler: Scheduler.default
	fraction_of_supported_ops: 1.0
	cpu_avx_type: avx2
	cpu_vnni: False


In [4]:
prompts = [
    "Finish the following function for computing a fibonacci sequence: \n\n fib(n):",
    "Write a function for filtering a list of integers to include only positive numbers:\n\nfilter(lst):",
    "Write a function for reversing a string:\n\ndef reverse_string(s):",
    "Write a function for checking if a word if a palindrome:\n\ndef is_palindrome(word):",
    "Write a function for sorting an array of integers:\n\ndef merge_sort(arr):",
]

def make_batch(id, prompt):
    return Batch(
        id=id,
        requests=[Request(id=id, prompt=prompt)]
    )

class PrefillQueue:
    def __init__(self, prompts):
        self.queue = {
            idx: PrefillRequest(batch=make_batch(id=idx, prompt=prompt))
            for idx, prompt in enumerate(prompts)
        }

    def pop(self):
        keys = list(self.queue.keys())
        if len(keys) == 0:
            return None
        else:
            return self.queue.pop(keys[0])

In [5]:
service.ClearCache()

# prefill queue
prefill_queue = PrefillQueue(prompts)

# cached batches
cached_batches = []

# generated
generated_text = {}

def prefill(request):
    generation, cached_batch = service.Prefill(request)
    
    assert request.batch.requests[0].id == generation.request_id
    assert generation.request_id not in generated_text.keys()
    
    generated_text[generation.request_id] = request.batch.requests[0].prompt + generation.generated_text

    return cached_batch

def decode(request):
    for cached_batch in request.batches:
        for request_id in cached_batch.request_ids:
            assert request_id in generated_text.keys()

    generations, cached_batch = service.Decode(request)
    if cached_batch is None:
        print("All requests done!\n\n")
        return None
    
    active_request_ids = []
    stopped_request_ids = []
    
    for generation in generations:
        assert generation.request_id in generated_text.keys()

        # if text is None, we stopped
        if generation.generated_text is None:
            print(f"Request {generation.request_id} is done!")
            stopped_request_ids.append(generation.request_id)
            
        else:
            generated_text[generation.request_id] += generation.generated_text
            active_request_ids.append(generation.request_id)
        
    # if any stopped, return this
    if len(stopped_request_ids) > 0:
        cached_batch = service.FilterBatch(FilterBatchRequest(
            batch_id=cached_batch.batch_id,
            request_ids=active_request_ids,
        ))
        
    return cached_batch

# run a prefille
queue_not_empty = True
while queue_not_empty:
    prefill_request = prefill_queue.pop()
    if prefill_request is not None:
        cached_batch = prefill(prefill_request)
        cached_batches.append(cached_batch)
    else:
        queue_not_empty = False
    
    # run a few decodes
    for _ in range(5):
        cached_batches = [decode(DecodeRequest(cached_batches))]

NameError: name 'Batch' is not defined

In [ ]:
# run a few decodes
for _ in range(100):
    cached_batch = decode(DecodeRequest(cached_batches))
    if cached_batch is None:
        break
    cached_batches = [cached_batch]
    
for idx, value in generated_text.items():
    print(f"INDEX = {idx}:")
    print(value)
    print("\n")

print(cached_batches)

## Example DeepSparseCausalLMBatch

In [1]:
from server.deepsparse.deepsparse_causal_lm import DeepSparseCausalLMBatch, DeepSparseCausalLM
from server.deepsparse.deepsparse_requests import Request, Batch
from transformers import AutoTokenizer

tokenizer_path = "/home/robertgshaw/.cache/sparsezoo/neuralmagic/codegen_mono-350m-bigpython_bigquery_thepile-base/deployment"
onnx_path = "/home/robertgshaw/.cache/sparsezoo/neuralmagic/codegen_mono-350m-bigpython_bigquery_thepile-base/model.onnx/model.onnx"
tokenizer = AutoTokenizer.from_pretrained(tokenizer_path)

2023-08-22 01:33:22 deepsparse.transformers WARNING  The neuralmagic fork of transformers may not be installed. It can be installed via `pip install nm_transformers`


In [2]:
ds_model = DeepSparseCausalLM(
    tokenizer_path=tokenizer_path,
    model_path=onnx_path
)

Using pad_token, but it is not set yet.
2023-08-22 01:33:25 deepsparse.transformers.utils.helpers INFO     Overwriting in-place the input shapes of the transformer model at /home/robertgshaw/.cache/sparsezoo/neuralmagic/codegen_mono-350m-bigpython_bigquery_thepile-base/model.onnx/model.onnx
DeepSparse, Copyright 2021-present / Neuralmagic, Inc. version: 1.6.0.20230815 COMMUNITY | (134dba40) (release) (optimized) (system=avx2, binary=avx2)


deepsparse.engine.Engine:
	onnx_file_path: /home/robertgshaw/.cache/sparsezoo/neuralmagic/codegen_mono-350m-bigpython_bigquery_thepile-base/model.onnx/model.onnx
	batch_size: 1
	num_cores: 8
	num_streams: 1
	scheduler: Scheduler.default
	fraction_of_supported_ops: 1.0
	cpu_avx_type: avx2
	cpu_vnni: False


2023-08-22 01:33:49 deepsparse.transformers.utils.helpers INFO     Overwriting in-place the input shapes of the transformer model at /home/robertgshaw/.cache/sparsezoo/neuralmagic/codegen_mono-350m-bigpython_bigquery_thepile-base/model.onnx/model.onnx


deepsparse.engine.Engine:
	onnx_file_path: /home/robertgshaw/.cache/sparsezoo/neuralmagic/codegen_mono-350m-bigpython_bigquery_thepile-base/model.onnx/model.onnx
	batch_size: 1
	num_cores: 8
	num_streams: 1
	scheduler: Scheduler.default
	fraction_of_supported_ops: 1.0
	cpu_avx_type: avx2
	cpu_vnni: False


In [3]:
sequence = "Finish the following function for computing a fibonacci sequence: \n\n fib(n):"

def make_n_requests(n=1):
    requests = []
    for i in range(n):
        request = Request(
            id=i,
            prompt=sequence,
        )
        requests.append(request)
    return requests

batch_size = 2
batch = Batch(
    id=0,
    requests = make_n_requests(n=batch_size),
)

ds_batch = DeepSparseCausalLMBatch.from_batch(
    batch=batch,
    tokenizer=tokenizer, 
)

next_batch = ds_batch
for _ in range(64):
    # print(tokenizer.batch_decode(next_batch.input_ids_list[0]))
    generation, next_batch = ds_model.generate_token(next_batch)

for input_ids in next_batch.input_ids_list:
    print(tokenizer.batch_decode(input_ids)[0])

Using pad_token, but it is not set yet.


Finish the following function for computing a fibonacci sequence: 

 fib(n):

    if n == 0:
        return 0
    elif n == 1:
        return 1
    else:
        return fib(n-1) + fib(n-2)

# Call the function.
print(fib(5))

# This code
Finish the following function for computing a fibonacci sequence: 

 fib(n):

    if n == 0:
        return 0
    elif n == 1:
        return 1
    else:
        return fib(n-1) + fib(n-2)

# Call the function.
print(fib(5))

# This code


## Compare to DeepSparse Pipeline

In [134]:
multitoken_length = 4

def sample_token(logits):
    assert(logits.shape[0] == 1)        # assert b=1 for now
    return np.argmax(logits[0,-1,:])  
    
def prefill_pipeline(pipeline, tokens):
    num_tokens_processed = 0
    for engine_inputs in pipeline.engine_inputs_for_prefill(tokens):
        _, logits = pipeline.multitoken_engine(engine_inputs)
        num_tokens_processed += multitoken_length
    
    if num_tokens_processed > 0:
        pipeline.engine.transfer_cache_state(cache=pipeline.multitoken_engine.kv_cache)

    run_tokens = [] if num_tokens_processed == 0 else tokens[:num_tokens_processed]
    for token in tokens[num_tokens_processed:]:
        run_tokens.append(token)
        new_token, logits = pipeline.autoregressive_inference(run_tokens)
    return logits
    
pipeline._reset_engines_cache()
engine_inputs = pipeline.process_inputs(pipeline.parse_inputs(sequences=sequence))[0]
tokens = engine_inputs[0][engine_inputs[1].nonzero()].tolist()

logits = prefill_pipeline(pipeline, tokens)
# print(logits)
tokens.append(sample_token(logits))

for _ in range(64):
    _, logits = pipeline.autoregressive_inference(tokens)
    # print(logits)
    tokens.append(sample_token(logits))

print(pipeline.tokenizer.decode(tokens))

Finish the following function for computing a fibonacci sequence: 

 fib(n):

    if n == 0:
        return 0
    elif n == 1:
        return 1
    else:
        return fib(n-1) + fib(n-2)

# Call the function.
print(fib(5))

# This code is


In [ ]:
from deepsparse import Pipeline
pipeline = Pipeline.create(
    task="text-generation", 
    model_path="zoo:nlg/text_generation/codegen_mono-350m/pytorch/huggingface/bigpython_bigquery_thepile/base-none",
    use_deepsparse_cache=False,
    prompt_processing_sequence_length=4,
    max_generated_tokens=64,
    sequence_length=128
)

In [101]:
next_batch.input_ids_list[0].shape

(1, 18)

In [97]:
type(next_batch.input_ids_list[0])

numpy.ndarray

In [98]:
tokenizer.decode(next_batch.input_ids_list[0])

TypeError: argument 'ids': 'list' object cannot be interpreted as an integer

In [80]:
print(next_batch.input_ids_list[1].shape)

(1, 20)


Using pad_token, but it is not set yet.
2023-08-21 18:14:09 deepsparse.transformers.utils.helpers INFO     Overwriting in-place the input shapes of the transformer model at /home/robertgshaw/.cache/sparsezoo/neuralmagic/codegen_mono-350m-bigpython_bigquery_thepile-base/model.onnx/model.onnx
DeepSparse, Copyright 2021-present / Neuralmagic, Inc. version: 1.6.0.20230815 COMMUNITY | (134dba40) (release) (optimized) (system=avx2, binary=avx2)


deepsparse.engine.Engine:
	onnx_file_path: /home/robertgshaw/.cache/sparsezoo/neuralmagic/codegen_mono-350m-bigpython_bigquery_thepile-base/model.onnx/model.onnx
	batch_size: 1
	num_cores: 8
	num_streams: 1
	scheduler: Scheduler.default
	fraction_of_supported_ops: 1.0
	cpu_avx_type: avx2
	cpu_vnni: False


2023-08-21 18:14:33 deepsparse.transformers.utils.helpers INFO     Overwriting in-place the input shapes of the transformer model at /home/robertgshaw/.cache/sparsezoo/neuralmagic/codegen_mono-350m-bigpython_bigquery_thepile-base/model.onnx/model.onnx


deepsparse.engine.Engine:
	onnx_file_path: /home/robertgshaw/.cache/sparsezoo/neuralmagic/codegen_mono-350m-bigpython_bigquery_thepile-base/model.onnx/model.onnx
	batch_size: 1
	num_cores: 8
	num_streams: 1
	scheduler: Scheduler.default
	fraction_of_supported_ops: 1.0
	cpu_avx_type: avx2
	cpu_vnni: False


In [73]:
import numpy as np
a = np.array([np.arange(10)]*2)
b = np.array([np.arange(10)]*1)

print(b[:,-1:])
print(b.shape)

[[9]]
(1, 10)


(1, 18)
(1, 19)


In [ ]:
import deepsparse
import torch
from transformers import AutoTokenizer
from server.text_generation_server.models.deepsparse_causal_lm import DeepSparseCausalLMBatch
from text_generation_server.utils import NextTokenChooser, StoppingCriteria, Sampling, StopSequenceCriteria

from server.text_generation_server.pb.generate_pb2 import (
    Batch,    
    Request, 
    NextTokenChooserParameters, 
    StoppingCriteriaParameters
)

In [ ]:
model_path = "/home/robertgshaw/.cache/sparsezoo/neuralmagic/codegen_mono-350m-bigpython_bigquery_thepile-base/deployment"
onnx_path = "/home/robertgshaw/.cache/sparsezoo/neuralmagic/codegen_mono-350m-bigpython_bigquery_thepile-base/model.onnx/model.onnx"
tokenizer = AutoTokenizer.from_pretrained(model_path)

In [ ]:
next_token_chooser = NextTokenChooser(
    watermark=False,
    temperature=1.0,
    repetition_penalty=0.0,
    top_k=None,
    top_p=None,
    typical_p=None,
    do_sample=False,
    seed=0,
    device="cpu",
)

In [ ]:
stopping_crtieria=StoppingCriteria(
    eos_token_id=tokenizer.eos_token_id,
    stop_sequence_criterias=[],
    max_new_tokens=20,
    ignore_eos_token=False,
)

In [ ]:
sequence = "Finish the following function for computing a fibonacci sequence: \n\n fib(n):"

In [ ]:
request_idx = 0

max_new_tokens = 64

parameters = NextTokenChooserParameters(
    watermark=False,
    temperature=1.0,
    repetition_penalty=0.0,
    do_sample=False,
    typical_p=1.0,
    top_k = 0,
    top_p = 1.0,
)

stopping_parameters = StoppingCriteriaParameters(
    max_new_tokens=max_new_tokens
)

def make_n_requests(n=1):
    requests = []
    for i in range(n):
        request = Request(
            id=request_idx,
            inputs=sequence,
            truncate=False,
            parameters=parameters,
            stopping_parameters=stopping_parameters,
            prefill_logprobs=False
        )
        requests.append(request)
    return requests

batch_size = 2
requests = make_n_requests(n=batch_size)

batch = Batch(
    id = 0,
    requests = requests,
    size=len(requests),
)

ds_batch = DeepSparseCausalLMBatch.from_pb(
    pb=batch, 
    tokenizer=tokenizer, 
    dtype=torch.float32,
    device="cpu"
)

In [ ]:
import os
os.environ["TOKENIZERS_PARALLELISM"] = "True"
os.environ["WAND_OPT_FLAGS"] = "default,~pyramids"

In [ ]:
from server.text_generation_server.models.deepsparse_model import DeepSparseDecoderModel, DeepSparsePastKeyValues
from transformers import AutoTokenizer

model_path = "/home/robertgshaw/.cache/sparsezoo/neuralmagic/codegen_mono-350m-bigpython_bigquery_thepile-base/deployment"
onnx_path = "/home/robertgshaw/.cache/sparsezoo/neuralmagic/codegen_mono-350m-bigpython_bigquery_thepile-base/model.onnx/model.onnx"
tokenizer = AutoTokenizer.from_pretrained(model_path)

In [ ]:
ds_decoder_model = DeepSparseDecoderModel(
    onnx_file_path = onnx_path,
    sequence_length = 128,
    multitoken_length = 4,
    # singletoken_engine = ds_decoder_model.singletoken_engine,
    # multitoken_engine = ds_decoder_model.multitoken_engine
)

In [ ]:
from deepsparse import Pipeline
pipeline = Pipeline.create(
    task="text-generation", 
    model_path="zoo:nlg/text_generation/codegen_mono-350m/pytorch/huggingface/bigpython_bigquery_thepile/base-none",
    use_deepsparse_cache=False,
    prompt_processing_sequence_length=4,
    max_generated_tokens=64,
    sequence_length=128
)

In [ ]:
sequence = "Finish the following function for computing a fibonacci sequence: \n\n fib(n):"

In [ ]:
import numpy as np
def sample_token(logits):
    assert(logits.shape[0] == 1)
    return np.argmax(logits[0,-1,:])

In [ ]:
print("testing DeepSparseDecoderModel:\n")

engine_inputs = pipeline.process_inputs(pipeline.parse_inputs(sequences=sequence))[0]
tokens = engine_inputs[0][engine_inputs[1].nonzero()].tolist()

past_key_values = DeepSparsePastKeyValues()
logits, past_key_values = ds_decoder_model.prefill(tokens, past_key_values)
tokens.append(sample_token(logits))

while len(tokens) < 64:
    logits, past_key_values = ds_decoder_model.decode(tokens, past_key_values)
    tokens.append(sample_token(logits))

print(pipeline.tokenizer.decode(tokens))

In [ ]:
multitoken_length = 4

def prefill_pipeline(pipeline, tokens):
    num_tokens_processed = 0
    for engine_inputs in pipeline.engine_inputs_for_prefill(tokens):
        _, logits = pipeline.multitoken_engine(engine_inputs)
        num_tokens_processed += multitoken_length

    if num_tokens_processed > 0:
        pipeline.engine.transfer_cache_state(cache=pipeline.multitoken_engine.kv_cache)

    run_tokens = [] if num_tokens_processed == 0 else tokens[:num_tokens_processed]
    for token in tokens[num_tokens_processed:]:
        run_tokens.append(token)
        new_token, logits = pipeline.autoregressive_inference(run_tokens)
    return logits
    
pipeline._reset_engines_cache()
engine_inputs = pipeline.process_inputs(pipeline.parse_inputs(sequences=sequence))[0]
tokens = engine_inputs[0][engine_inputs[1].nonzero()].tolist()

logits = prefill_pipeline(pipeline, tokens)
tokens.append(sample_token(logits))

while len(tokens) < 64:
    _, logits = pipeline.autoregressive_inference(tokens)
    tokens.append(sample_token(logits))

print(pipeline.tokenizer.decode(tokens))

In [ ]:
print(f"{sequence}{pipeline(sequences=sequence).sequences[0]}")